In [ ]:
# Import libraries:
import pandas as pd
import dt_ids7_export_utils as bh_utils
import elfys_functions as bh_elfys

# Get the name of the computer:
import socket
hostname = socket.gethostname()
if hostname == 'BHs-Desktop':
    path = 'E:\\'
elif hostname == 'BH-XPS':
    path = 'G:\\'
else:
    path = 'D:\\'

del hostname
# Import the data:
df_ids7 = pd.read_excel(path + 'RRH ELfys Ids7.xlsx')
df_dt = pd.read_excel(path + 'DoseTrack RRH Elfys reduced.xlsx')

del path


# Run all cleanups, filters and checks:
df_ids7 = bh_utils.run_all_cleanup_filters_and_checks(df_ids7, df_dt, True)
# Check for accession numbers that exists in DoseTrack but not in Ids7:
df_dt = bh_utils.check_accession_dt_vs_ids7(df_dt, df_ids7, True)

In [ ]:
# First we perform mapping on selected procedures:
df_ids7 = bh_elfys.map_procedures(df_ids7, True)

In [ ]:
print(len(df_dt[df_dt['Henvisning_i_ids7'] == False]['Accession Number'].drop_duplicates()))

print(len(df_dt[df_dt['Henvisning_i_ids7'] == True]['Accession Number'].drop_duplicates()))

In [ ]:
# Make a dataframe to collect the data to plot:

df_ids7_to_merge = df_ids7[df_ids7['Henvisning_i_dt'] == True].groupby('Henvisnings-ID').agg({'Pasient': 'first', 
                                                 'Kjønn': 'first',
                                                 'Henvisnings-ID': 'first', 
                                                 'Beskrivelse': bh_utils.concatenate_protocol})

df_dt_to_merge = df_dt[df_dt['Henvisning_i_ids7'] == True].groupby('Accession Number').agg({'Accession Number': 'first',
                                                    'Study Date': 'first',
                                                    'Age (Years)': 'first',
                                                    'DAP Total (Gy*cm2)': 'sum',
                                                    'CAK (mGy)': 'sum',
                                                    'F+A Time (s)': 'sum',
                                                    'Modality Room': 'first'}).reset_index()

data = pd.merge(df_ids7_to_merge, df_dt_to_merge, how='outer', left_on='Henvisnings-ID', right_on='Accession Number')



In [ ]:
df_ids7['Beskrivelse'].str.contains('RGA Cor Ablasjon').any()

In [ ]:
# Check for patients with multiple bookings on the same time, with different accession numbers:
bh_utils.check_patents_with_multiple_bookings_on_same_time_with_different_accession(df_ids7)

In [ ]:
# Check for patients with multiple bookings on the same day but not the same time, with different accession numbers:
bh_utils.check_patents_with_multiple_bookings_on_same_day_with_different_accession(df_ids7)